In [8]:
### Don't mind about this 
import warnings
warnings.filterwarnings('ignore')
###
%matplotlib inline 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, classification_report
import math
import ast
import re
from collections import defaultdict

In [9]:
class Layer:
    def sigmoid(self,z):
        return (1/(1+np.exp(-1*z)))

    def _diff_sigmoid(self,z):
        return self.sigmoid(z)*(1-self.sigmoid(z))


    def _tanh(self, z):
        return np.tanh(z)

    def _diff_tanh(self, z):
        return 1-np.square(self._tanh(z))

    def relu(self,z):
        return np.maximum(0,z)

    def _diff_relu(self, z):
        a= np.zeros_like(z,dtype='int')
        a[z>0] = 1
        return a

    def softmax(self,z):
        exp = np.exp(z)
        tot= exp.sum(axis=0)
        t= exp/tot
        return t
    
    def _diff_softmax(self, z,y):
        yhat_r = self.softmax(z)
        one_yi = y *-1*(1-yhat_r)
        z=(1-y)*yhat_r
        return one_yi +z

    
    def __init__(self,n_neurons,n_inputs, activation='relu'):
        self.n_neurons = n_neurons #number of neurons 
        self.n_inputs = n_inputs #past number of samples
        self.activation_name = activation

        self.W= np.random.randn(self.n_neurons,self.n_inputs)*np.sqrt(2/self.n_inputs)
        self.b= np.random.randn(self.n_neurons,1)*np.sqrt(2/self.n_inputs)

        self.dW = np.zeros_like(self.W)
        self.db = np.zeros_like(self.b)
    
    def _set_target(self, t):
        self.target=t

    def forward(self,Ai):
        z = np.dot(self.W,Ai)+ self.b
        if (self.activation_name=='relu'):
            A = self.relu(z)
        elif(self.activation_name=='softmax'):
            A = self.softmax(z)
        else:
            A = self._tanh(z)
        self.Ai = Ai
        self.Z = z
        return A
    
    def backward(self,inp):
        if (self.activation_name=='relu'):
            act_diff = self._diff_relu(self.Z)
        elif(self.activation_name=='softmax'):
            act_diff = self._diff_softmax(self.Z,self.target)
        else:
            act_diff = self._diff_tanh(self.Z) 
        e = np.ones((self.Ai.shape[1],1))
        bet= inp * act_diff
        self.dW= self.dW+ np.dot(bet,self.Ai.T)
        self.db= self.db+ np.dot(bet,e)
        return np.dot(self.W.T, bet)
    
    def zeroing_diff(self):
        self.dW = np.zeros_like(self.W)
        self.db = np.zeros_like(self.b)

In [10]:
class NN:
    def __init__(self,lr):
        self.alpha = lr
        self.layers = []
        self.t_loss=[]
        self.v_loss=[]

    def forward(self,X):
        a = X   
        for layer in self.layers:
            a = layer.forward(a)
        return a
    
    def backward(self,inp):
        gd = inp
        for layer in self.layers[::-1]:
            gd = layer.backward(gd)
            
    def add_layer(self,n_neurons,n_inputs,act):
        self.layers.append(Layer(n_neurons,n_inputs,act))
    
    def loss(self, y_hat, y):
        onehotY= y
        yhat_r = np.max(onehotY*y_hat, axis=0,keepdims=True)
        return (1/(y.shape[1]))*-1*np.sum(np.log(yhat_r))
    
    def fit(self,x_train,y_train):
        M = x_train.shape[1]
        for i in range(1):
            y_hat = self.forward(x_train)      
            gd=self.loss(y_train, y_hat)
            print("Training loss: {}....".format(gd))
            self.t_loss.append(gd)
            self.layers[-1]._set_target(y_train)
            self.backward(1)   
            # apply GD
            for i in range(len(self.layers)):
                self.layers[i].W = self.layers[i].W - (self.alpha * (self.layers[i].dW/M))
                self.layers[i].b = self.layers[i].b - (self.alpha * (self.layers[i].db/M))
                
            for layer in self.layers:
                layer.zeroing_diff()
        return self.t_loss
                
    def predict(self,data): 
        y_hat= self.forward(data)
        return y_hat

    def save_weights(self, path):
        params= []
        for layer in self.layers:
            layer_param= [layer.W , layer.b]
            params.append(layer_param)
        file = open(path, 'wb')
        pickle.dump(params, file)
        file.close()

# WORD2VEC ENCODING SKIPGRAM

In [11]:
df_all= pd.read_csv('All_data.csv')
df_all
titles_corpus= []
for title in df_all["dic_words"][:]:
    new_list = list(ast.literal_eval(title).keys())
    if (new_list != []):
        titles_corpus.append(new_list)

In [12]:
all_words = sorted(list({x for l in titles_corpus for x in l}))
word_index = dict((w, i) for i, w in enumerate(all_words))

def get_one_hot(word, size):
    word_vec = np.zeros(size)
    w_idx = word_index[word]
    word_vec[w_idx] = 1
    return word_vec

In [13]:
def get_training_data(titles_corpus, window):
    data=[]
    for sentence in titles_corpus:
        if (len(sentence) >= 3):
            for i in range (len(sentence)):
                w_enc= get_one_hot(sentence[i], len(all_words))
                context_words = []
                for j in range(i-window, i+window+1):
                    if j!=i and j<=len(sentence)-1 and j>=0:
                        context_words.append(sentence[j])
                for c in context_words:
                    c_hotenc = get_one_hot(c, len(all_words))
                    data.append([w_enc,c_hotenc])
    return np.array(data)

In [40]:
W2V = NN(0.01)
W2V.add_layer(100,3125,'relu')
W2V.add_layer(3125,100,'softmax')
j=1
for e in range (20):
    print("###########epoch#######", j )
    for i in range(0, len(titles_corpus), 3000):
        if(i+3000> len(titles_corpus)):
            training= get_training_data(titles_corpus[i:len(titles_corpus)], 3)
        else:
            training= get_training_data(titles_corpus[i:i+3000], 3)
        X_t=training[:,0,:]
        y_t=training[:,1,:]
        X_t_arr=np.array(X_t).T
        Y_t_arr=np.array(y_t).T
        W2V.fit(X_t_arr, Y_t_arr)
    j+=1

###########epoch####### 1
Training loss: 8.056057898894611....
Training loss: 8.056171309817975....
Training loss: 8.058084173903493....
Training loss: 8.057922314912007....
Training loss: 8.058757734351987....
Training loss: 8.058479684857756....
Training loss: 8.056100148834686....
Training loss: 8.056970339879848....
Training loss: 8.058200026956968....
Training loss: 8.061039604602383....
Training loss: 8.058186138467878....
Training loss: 8.059832972496965....
Training loss: 8.059421754190193....
Training loss: 8.056722147107681....
Training loss: 8.055643132719753....
Training loss: 8.056334977769943....
Training loss: 8.0559905500657....
Training loss: 8.056285055155747....
Training loss: 8.056832821064715....
Training loss: 8.057701738791819....
Training loss: 8.056338440923005....
Training loss: 8.057781787410974....
Training loss: 8.056426571556324....
Training loss: 8.056183985273174....
Training loss: 8.054631121892044....
Training loss: 8.058535726702889....
Training loss:

Training loss: 8.055168001403702....
Training loss: 8.055956082478234....
Training loss: 8.054818417290454....
Training loss: 8.054588516982989....
Training loss: 8.052537131310437....
Training loss: 8.057013225186843....
Training loss: 8.058000171704716....
Training loss: 8.05350007002831....
Training loss: 8.05546022081079....
Training loss: 8.056456578829298....
Training loss: 8.056257700035715....
Training loss: 8.052704972507268....
Training loss: 8.05481762432386....
Training loss: 8.057222029172351....
Training loss: 8.055255213254378....
Training loss: 8.053402611545913....
Training loss: 8.059758087319064....
Training loss: 8.057480037821769....
Training loss: 8.052980122748597....
Training loss: 8.050637542050545....
Training loss: 8.056465495678935....
Training loss: 8.057690871038739....
Training loss: 8.057537334254588....
Training loss: 8.056122217448491....
Training loss: 8.055427283369765....
Training loss: 8.05936770193574....
Training loss: 8.053552506363259....
Train

Training loss: 8.055861310807655....
Training loss: 8.052056081358646....
Training loss: 8.05142214384874....
Training loss: 8.051385577066148....
Training loss: 8.050907169861313....
Training loss: 8.054942861461832....
Training loss: 8.052046225314703....
Training loss: 8.053331382137493....
Training loss: 8.054397329808685....
Training loss: 8.052769374159688....
Training loss: 8.05317142067788....
Training loss: 8.052550911157518....
Training loss: 8.05312022507788....
Training loss: 8.053069172872275....
Training loss: 8.057082181150161....
Training loss: 8.049928191553159....
Training loss: 8.051917398154307....
Training loss: 8.04810433195457....
Training loss: 8.050979735866004....
Training loss: 8.053591765692184....
Training loss: 8.053778340406089....
Training loss: 8.048747283588451....
Training loss: 8.053580532089224....
Training loss: 8.050387067498281....
Training loss: 8.057312666278676....
Training loss: 8.052270757720486....
Training loss: 8.049990862523671....
Train

Training loss: 8.051644736907006....
Training loss: 8.05274433517165....
Training loss: 8.048865310957291....
Training loss: 8.052345263080896....
Training loss: 8.045280948894298....
Training loss: 8.050261851657655....
Training loss: 8.049100663064708....
Training loss: 8.05187411813049....
Training loss: 8.052532098828284....
Training loss: 8.049682055586503....
Training loss: 8.052547630493269....
Training loss: 8.05067545112261....
Training loss: 8.051607313112196....
Training loss: 8.051091939287852....
Training loss: 8.04886127844926....
Training loss: 8.051352982919099....
Training loss: 8.05007653235047....
Training loss: 8.047750867518369....
Training loss: 8.049001586841605....
Training loss: 8.047177569776544....
Training loss: 8.052676028992378....
Training loss: 8.051802202668737....
Training loss: 8.050648183734461....
Training loss: 8.045280463021387....
Training loss: 8.048818354727663....
Training loss: 8.047590320294503....
Training loss: 8.049905914786319....
Traini

Training loss: 8.043530361460563....
Training loss: 8.048401883771149....
Training loss: 8.047934623682325....
Training loss: 8.046126674290209....
Training loss: 8.04820367011913....
Training loss: 8.049819851336236....
Training loss: 8.047297956596532....
Training loss: 8.043294933451229....
###########epoch####### 6
Training loss: 8.045835926681939....
Training loss: 8.045898178367331....
Training loss: 8.047204011510258....
Training loss: 8.047452129553541....
Training loss: 8.048096791827586....
Training loss: 8.047807022941505....
Training loss: 8.04533779787438....
Training loss: 8.046492242993011....
Training loss: 8.048320105428806....
Training loss: 8.050549650123425....
Training loss: 8.048157247240745....
Training loss: 8.049925965590921....
Training loss: 8.049257928279372....
Training loss: 8.046202705658846....
Training loss: 8.045111501492174....
Training loss: 8.045498465815246....
Training loss: 8.045704216457684....
Training loss: 8.045738556558122....
Training loss:

...

Training loss: 8.025455747129838....
Training loss: 8.025505849585379....
Training loss: 8.024466339084945....
Training loss: 8.027559048016002....
Training loss: 8.029994040055573....
Training loss: 8.026787340433756....
Training loss: 8.026236777157836....
Training loss: 8.025011386136555....
Training loss: 8.0254629757729....
Training loss: 8.024709243340332....
Training loss: 8.026148686608522....
Training loss: 8.026232928527724....
Training loss: 8.022361383541483....
Training loss: 8.027954159267354....
Training loss: 8.027349738696026....
Training loss: 8.025437641021629....
Training loss: 8.02738761084792....
Training loss: 8.028931889293505....
Training loss: 8.025518617983224....
Training loss: 8.022394600475296....
###########epoch####### 16
Training loss: 8.025581199302284....
Training loss: 8.02498760184772....
Training loss: 8.025855452435348....
Training loss: 8.026796566814776....
Training loss: 8.026748165552393....
Training loss: 8.026557295561384....
Training loss: 

Training loss: 8.022938665078136....
Training loss: 8.02058129863545....
Training loss: 8.02090645692741....
Training loss: 8.025382329526064....
Training loss: 8.017339227025996....
Training loss: 8.021865947149886....
Training loss: 8.023849486909612....
Training loss: 8.020688372569513....
Training loss: 8.024957866629453....
Training loss: 8.025278653495834....
Training loss: 8.024147946971626....
Training loss: 8.026428360013117....
Training loss: 8.023509491049554....
Training loss: 8.022101910058606....
Training loss: 8.02777803357444....
Training loss: 8.024966835464008....
Training loss: 8.023406837702861....
Training loss: 8.021950786792242....
Training loss: 8.019307282317053....
Training loss: 8.02443546246484....
Training loss: 8.02381552521776....
Training loss: 8.023526562522555....
Training loss: 8.023348753585127....
Training loss: 8.022502264154529....
Training loss: 8.02079758544069....
Training loss: 8.026204487090194....
Training loss: 8.026120218748366....
Trainin

Training loss: 8.023377074844657....
Training loss: 8.021232382793023....
Training loss: 8.026788917215473....
Training loss: 8.019607497940545....
Training loss: 8.020338023876581....
Training loss: 8.0255151119193....
Training loss: 8.018647899230615....
Training loss: 8.019137324191513....
Training loss: 8.02106296471265....
Training loss: 8.025823851682125....
Training loss: 8.022003671101105....
Training loss: 8.020962703621187....
Training loss: 8.020286398825123....
Training loss: 8.024252483783913....
Training loss: 8.018884324790264....
Training loss: 8.022673808259913....
Training loss: 8.018829146245801....
Training loss: 8.021963823654184....
Training loss: 8.0214194681455....
Training loss: 8.023511447100748....
Training loss: 8.024435920840794....
Training loss: 8.019164158619047....
Training loss: 8.0183673157927....
Training loss: 8.01743926081195....
Training loss: 8.019632766899225....
Training loss: 8.02497501940497....
Training loss: 8.020932224123175....
Training l

Training loss: 8.015851848930543....
Training loss: 8.02312759159087....
Training loss: 8.01810416004476....
Training loss: 8.01663915290747....
Training loss: 8.01649170516515....
Training loss: 8.019090977618479....
Training loss: 8.020409380077904....
Training loss: 8.024368097585697....
Training loss: 8.019311552787928....
Training loss: 8.021244815607014....
Training loss: 8.018109264298502....
Training loss: 8.013616477055885....
Training loss: 8.018328214850499....
Training loss: 8.018947043832153....
Training loss: 8.018837258614447....
Training loss: 8.019060194513838....
Training loss: 8.016063797448428....
Training loss: 8.016995783182258....
Training loss: 8.016742195101559....
Training loss: 8.01522017541847....
Training loss: 8.020318646824569....
Training loss: 8.020988810591332....
Training loss: 8.017424407563785....
Training loss: 8.020877563996718....
Training loss: 8.012707344248659....
Training loss: 8.017187584167642....
Training loss: 8.017744301331076....
Traini

Training loss: 8.011079013679625....
Training loss: 8.013343738995184....
Training loss: 8.014174643459171....
Training loss: 8.016018551354463....
Training loss: 8.01813943859955....
Training loss: 8.018348770018846....
Training loss: 8.012910010581802....
Training loss: 8.014672665758857....
Training loss: 8.013953718488857....
Training loss: 8.013647817422392....
Training loss: 8.013168025224166....
Training loss: 8.016262595736745....
Training loss: 8.018644715762894....
Training loss: 8.014826725829407....
Training loss: 8.014210079470374....
Training loss: 8.012999977102274....
Training loss: 8.013829207254657....
Training loss: 8.012736421753175....
Training loss: 8.014539584191805....
Training loss: 8.014231521311343....
Training loss: 8.010466725173263....
Training loss: 8.016527855457195....
Training loss: 8.015747796916555....
Training loss: 8.013858923584406....
Training loss: 8.015653354279454....
Training loss: 8.017176687769705....
Training loss: 8.013421410406771....
Tr

In [41]:
import pickle
W2V.save_weights("Emb_Weights.model")